In [1]:
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.model_selection import KFold

## Use retweet_count as dependent variable: without TF-IDF features

In [2]:
ldata = pd.read_csv("smalldata.csv")

In [3]:
ldata.head()

,Unnamed: 0,created_at,retweet_count,favorite_count,text,clean_text,length,hashtag,mention,president,sentiment,subjectivity,readability,retweet>100,favorite>200
0,0,Mon Sep 09 03:37:47 +0000 2019,36,152,"'The good, the bad, and the ugly side of havin...","'The good, the bad, and the ugly side of havin...",105.0,0.0,0.0,0.0,-0.233333,0.755556,5.1,0,0
1,1,Mon Sep 09 03:10:13 +0000 2019,43,163,"'In the Albanian capital of Tirana, the countr...","'In the Albanian capital of Tirana, the countr...",115.0,0.0,0.0,0.0,0.216667,0.358333,11.8,0,0
2,2,Mon Sep 09 02:50:02 +0000 2019,22,97,'The 17 Democratic candidates who are lagging ...,'The Democratic candidates who are lagging beh...,180.0,0.0,0.0,0.0,-0.050000,0.600000,17.3,0,0
3,4,Mon Sep 09 02:32:03 +0000 2019,247,531,"""President Trump's plan to secretly meet with ...","""President Trump's plan to secretly meet with ...",154.0,0.0,0.0,1.0,-0.100000,0.650000,12.2,1,1
4,8,Mon Sep 09 01:24:51 +0000 2019,116,621,"""For Paul Eng, trying to recreate his family's...","""For Paul Eng, trying to recreate his family's...",277.0,0.0,0.0,0.0,0.166667,0.333333,12.4,1,1


In [4]:
ldata = ldata.iloc[:, 1:] #delete first column

In [5]:
ldata_rc  = ldata.drop(['created_at', 'favorite_count','text','retweet>100','favorite>200','clean_text'], axis=1)

In [6]:
ldata_rc.head()

,retweet_count,length,hashtag,mention,president,sentiment,subjectivity,readability
0,36,105.0,0.0,0.0,0.0,-0.233333,0.755556,5.1
1,43,115.0,0.0,0.0,0.0,0.216667,0.358333,11.8
2,22,180.0,0.0,0.0,0.0,-0.050000,0.600000,17.3
3,247,154.0,0.0,0.0,1.0,-0.100000,0.650000,12.2
4,116,277.0,0.0,0.0,0.0,0.166667,0.333333,12.4


In [7]:
y = ldata_rc['retweet_count'] 
X = ldata_rc.loc[:, ldata_rc.columns != 'retweet_count']
from sklearn.linear_model import LinearRegression
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
lr = LinearRegression().fit(X_train, y_train)

In [8]:
print("Training set score: {:.2f}".format(lr.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lr.score(X_test, y_test)))

Training set score: 0.03
Test set score: 0.02


In [9]:
import statsmodels.api as sm

X_train_new = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_new ).fit()

model.summary() # get a complete summary of the model

/Users/chenluqi/anaconda3/envs/ml1/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          retweet_count   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     6.539
Date:                Fri, 20 Dec 2019   Prob (F-statistic):           1.24e-07
Time:                        22:44:28   Log-Likelihood:                -13800.
No. Observations:                1650   AIC:                         2.762e+04
Df Residuals:                    1642   BIC:                         2.766e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          215.2857    107.016      2.012      0.044       5.383     425.188
length          -0.1305      0.541     -0.241      0.810      -1.192       0.932
hashtag       -119.4082    242.249     -0.493      0.622    -594.559     355.742
mention        -73.9959     74.878     -0.988      0.323    -220.862      72.870
president      479.5068     74.443      6.441      0.000     333.493     625.520
sentiment       89.3043    124.216      0.719      0.472    -154.334     332.942
subjectivity     1.2293    105.068      0.012      0.991    -204.851     207.310
readability      1.3400      5.585      0.240      0.810      -9.615      12.295
==============================================================================
Omnibus:                     3135.147   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4610469.887
Skew:                          14.120   Prob(JB):                         0.00
Kurtosis:                     260.418   Cond. No.                     2.03e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Use retweet_count as dependent variable: with TF-IDF features

In [10]:
data_withTFIDF = pd.read_csv("bigdata.csv")
data_withTFIDF = data_withTFIDF.iloc[:, 1:] #delete first column
data_withTFIDF.head()

,created_at,retweet_count,favorite_count,text,clean_text,length,hashtag,mention,president,sentiment,...,zest,zhang,zimbabw,zimdancehal,zine,zoltan,zombi,zone,zoo,zuckerberg
0,Mon Sep 09 03:37:47 +0000 2019,36,152,"'The good, the bad, and the ugly side of havin...","'The good, the bad, and the ugly side of havin...",105.0,0.0,0.0,0.0,-0.233333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Mon Sep 09 03:10:13 +0000 2019,43,163,"'In the Albanian capital of Tirana, the countr...","'In the Albanian capital of Tirana, the countr...",115.0,0.0,0.0,0.0,0.216667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Mon Sep 09 02:50:02 +0000 2019,22,97,'The 17 Democratic candidates who are lagging ...,'The Democratic candidates who are lagging beh...,180.0,0.0,0.0,0.0,-0.050000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mon Sep 09 02:32:03 +0000 2019,247,531,"""President Trump's plan to secretly meet with ...","""President Trump's plan to secretly meet with ...",154.0,0.0,0.0,1.0,-0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Mon Sep 09 01:24:51 +0000 2019,116,621,"""For Paul Eng, trying to recreate his family's...","""For Paul Eng, trying to recreate his family's...",277.0,0.0,0.0,0.0,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
ldata_rct  = data_withTFIDF.drop(['created_at', 'favorite_count','text','retweet>100','favorite>200','clean_text','words'], axis=1)

In [12]:
ldata_rct.head()

,retweet_count,length,hashtag,mention,president,sentiment,subjectivity,readability,aaron,abaco,...,zest,zhang,zimbabw,zimdancehal,zine,zoltan,zombi,zone,zoo,zuckerberg
0,36,105.0,0.0,0.0,0.0,-0.233333,0.755556,5.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,43,115.0,0.0,0.0,0.0,0.216667,0.358333,11.8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22,180.0,0.0,0.0,0.0,-0.050000,0.600000,17.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,247,154.0,0.0,0.0,1.0,-0.100000,0.650000,12.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,116,277.0,0.0,0.0,0.0,0.166667,0.333333,12.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.linear_model import Lasso
y = ldata_rct['retweet_count']
X = ldata_rct.loc[:, ldata_rct.columns != 'retweet_count']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
lasso = Lasso().fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))

Training set score: 0.66
Test set score: 0.21


In [14]:
coeff_used = np.sum(lasso.coef_!=0)

In [15]:
coeff_used 

47

## Use favorite_count as dependent variable: without TF-IDF features

In [16]:
ldata_lc  = ldata.drop(['created_at', 'retweet_count','text','retweet>100','favorite>200','clean_text'], axis=1)
ldata_lc.head()

,favorite_count,length,hashtag,mention,president,sentiment,subjectivity,readability
0,152,105.0,0.0,0.0,0.0,-0.233333,0.755556,5.1
1,163,115.0,0.0,0.0,0.0,0.216667,0.358333,11.8
2,97,180.0,0.0,0.0,0.0,-0.050000,0.600000,17.3
3,531,154.0,0.0,0.0,1.0,-0.100000,0.650000,12.2
4,621,277.0,0.0,0.0,0.0,0.166667,0.333333,12.4


In [17]:
y = ldata_lc['favorite_count'] 
X = ldata_lc.loc[:, ldata_lc.columns != 'favorite_count']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
ll = LinearRegression().fit(X_train, y_train)
print("Training set score: {:.2f}".format(ll.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ll.score(X_test, y_test)))

Training set score: 0.01
Test set score: 0.01


In [18]:
X_train_new = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_new ).fit()

model.summary() # get a complete summary of the model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         favorite_count   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     3.549
Date:                Fri, 20 Dec 2019   Prob (F-statistic):           0.000859
Time:                        22:44:32   Log-Likelihood:                -15825.
No. Observations:                1650   AIC:                         3.167e+04
Df Residuals:                    1642   BIC:                         3.171e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          841.2837    365.197      2.304      0.021     124.983    1557.584
length          -0.7034      1.848     -0.381      0.703      -4.327       2.921
hashtag       -345.1339    826.687     -0.417      0.676   -1966.605    1276.337
mention       -202.1338    255.523     -0.791      0.429    -703.320     299.052
president     1193.6604    254.040      4.699      0.000     695.384    1691.937
sentiment      489.9320    423.891      1.156      0.248    -341.493    1321.357
subjectivity    11.7890    358.547      0.033      0.974    -691.469     715.047
readability     -7.3155     19.059     -0.384      0.701     -44.699      30.068
==============================================================================
Omnibus:                     3442.985   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8780139.170
Skew:                          17.264   Prob(JB):                         0.00
Kurtosis:                     358.695   Cond. No.                     2.03e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Use favorite_count as dependent variable: with TF-IDF features

In [19]:
ldata_lct  = data_withTFIDF.drop(['created_at', 'retweet_count','retweet>100','favorite>200','text','clean_text','words'], axis=1)
ldata_lct.head()

,favorite_count,length,hashtag,mention,president,sentiment,subjectivity,readability,aaron,abaco,...,zest,zhang,zimbabw,zimdancehal,zine,zoltan,zombi,zone,zoo,zuckerberg
0,152,105.0,0.0,0.0,0.0,-0.233333,0.755556,5.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,163,115.0,0.0,0.0,0.0,0.216667,0.358333,11.8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,97,180.0,0.0,0.0,0.0,-0.050000,0.600000,17.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,531,154.0,0.0,0.0,1.0,-0.100000,0.650000,12.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,621,277.0,0.0,0.0,0.0,0.166667,0.333333,12.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
y = ldata_lct['favorite_count']
X = ldata_lct.loc[:, ldata_lct.columns != 'favorite_count']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
lasso = Lasso().fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))

Training set score: 0.88
Test set score: -0.01


In [21]:
coeff_used = np.sum(lasso.coef_!=0)
coeff_used

198